In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve

import numpy as np

In [ ]:
def score(model, X_test, y_test):   
    y_pred = model.predict(X_test)

    print('Recall: %.3f \nPrecision: %.3f\nF1: %.3f \nAccuracy: %.3f \nROC AUC: %.3f' % \
        (recall_score(y_test, y_pred),
        precision_score(y_test, y_pred), 
        f1_score(y_test, y_pred), 
        accuracy_score(y_test, y_pred),
        roc_auc_score(y_test, model.predict_proba(X_test)[:,1])))

In [ ]:
def fit_and_score(model, X, y, X_test, y_test):
    model.fit(X, y)
    
    score(model, X_test, y_test)

In [ ]:
def confusion_diagram_simple(y_test, y_pred, print_percents=True):
  confusion = confusion_matrix(y_test, y_pred)
  plt.figure(dpi=100)
  palette = sns.color_palette("ch:start=0.1,rot=-0.1", as_cmap=True)
  ax = sns.heatmap(confusion, cmap=palette, annot=True, square=True, fmt='d', cbar=False, linewidths=0.1, linecolor='grey',
            xticklabels=['no claim', 'claim'],
            yticklabels=['no claim', 'claim']);
  ax.spines['top'].set_color('grey')
  ax.spines['top'].set_visible(True) 
  ax.spines['right'].set_color('grey')
  ax.spines['right'].set_visible(True) 
  plt.xlabel('prediction')
  plt.ylabel('actual')
  plt.title('confusion matrix \n')

  if print_percents:
    group_percents(confusion)

  

In [2]:
def group_percents(confusion):
    if 0 not in confusion:
      no_claim_pred_percent = confusion[1][0] / (confusion[0][0] + confusion[1][0]) * 100
      no_claim_pred_percent

      claim_pred_percent = confusion[1][1] / (confusion[0][1] + confusion[1][1]) * 100
      claim_pred_percent

      print('Percent claims in predicted no claim group: %.2f%% \n\
Percent claims in predicted claim group: %.2f%% \n' % (no_claim_pred_percent,claim_pred_percent))
         
      payout_per_low_risk_member = confusion[1][0] * 7000 / (confusion[0][0] + confusion[1][0])
      payout_per_high_risk_member = confusion[1][1] * 7000 / (confusion[0][1] + confusion[1][1])

      print('Av payout per low risk member: $%.2f \n\
Av payout per high risk member: $%.2f \n' % (payout_per_low_risk_member,payout_per_high_risk_member))
      
      print('Total Policies: %.2f \nTotal Claims: %.2f \nOverall Percent claims %.2f%%\n\n' % \
      (np.sum(confusion),
      np.sum(confusion[1]),
      np.sum(confusion[1]) / np.sum(confusion) * 100 ))

In [ ]:
def confusion_diagram(X_test, y_test, model):
  y_pred = model.predict(X_test)
  
  confusion_diagram_simple(y_test, y_pred)

In [ ]:
def precision_recall_diagram(X_test, y_test, model):
  precision_curve, recall_curve, threshold_curve = precision_recall_curve(y_test, model.predict_proba(X_test)[:,1] )

  plt.figure(dpi=100)
  plt.plot(threshold_curve, precision_curve[1:],label='precision')
  plt.plot(threshold_curve, recall_curve[1:], label='recall')
  plt.legend(loc='upper right')
  plt.xlabel('Threshold (above this probability, label as \'filed claim\')');
  plt.title('Precision and Recall Curves');

In [ ]:
def roc_diagram(X_test, y_test, model):
  fpr, tpr, thresholds = roc_curve(y_test, model.predict_proba(X_test)[:,1])

  plt.plot(fpr, tpr,lw=2)
  plt.plot([0,1],[0,1],c='violet',ls='--')
  plt.xlim([-0.05,1.05])
  plt.ylim([-0.05,1.05])

  plt.xlabel('False positive rate')
  plt.ylabel('True positive rate')
  plt.title('ROC curve for predicting claims');
  print("ROC AUC score = ", roc_auc_score(y_test, model.predict_proba(X_test)[:,1]))

In [ ]:
import warnings
warnings.filterwarnings("ignore")

def threshold_diagram(X_test, y_test, model):
  thresh_ps = np.linspace(0,1,1000)
  model_test_probs = model.predict_proba(X_test)[:,1]
  f1_scores, prec_scores, rec_scores, acc_scores = [], [], [], []
  for p in thresh_ps:
      model_val_labels = model_test_probs >= p
      f1_scores.append(f1_score(y_test, model_val_labels))    
      prec_scores.append(precision_score(y_test, model_val_labels))
      rec_scores.append(recall_score(y_test, model_val_labels))
      acc_scores.append(accuracy_score(y_test, model_val_labels))
      
  plt.plot(thresh_ps, f1_scores)
  plt.plot(thresh_ps, prec_scores)
  plt.plot(thresh_ps, rec_scores)
  plt.plot(thresh_ps, acc_scores)

  plt.title('Metric Scores vs. Positive Class Decision Probability Threshold')
  plt.legend(['F1','Precision','Recall','Accuracy'], bbox_to_anchor=(1.05, 0), loc='lower left')
  plt.xlabel('P threshold')
  plt.ylabel('Metric score')

  best_f1_score = np.max(f1_scores) 
  best_thresh_p = thresh_ps[np.argmax(f1_scores)]

  print('Best F1 score %.3f at prob decision threshold >= %.3f' 
        % (best_f1_score, best_thresh_p))

In [ ]:
def eval_gini(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini